In [1]:
# !pip install faiss-cpu --no-cache
# !pip install opencv-python==4.8.0.76

import faiss
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from PIL import Image
import os
import shutil
import random
import zipfile

import torch
from torch import nn
from torchvision.models import resnet34
import torchvision.transforms as transforms
from tqdm import tqdm

# 0. Подготовка датасета

https://www.kaggle.com/datasets/vishweshsalodkar/wild-animals

В датасете есть изображения гепардов, ягуаров, леопардов, львов и тигров (примерно по 30 шт)

In [2]:
! kaggle datasets download -d vishweshsalodkar/wild-animals

# Распакуем архив
with zipfile.ZipFile("wild-animals.zip", 'r') as zip_ref:
  zip_ref.extractall("wild-animals")

Dataset URL: https://www.kaggle.com/datasets/vishweshsalodkar/wild-animals
License(s): other
 95% 7.00M/7.37M [00:00<00:00, 72.9MB/s]
100% 7.37M/7.37M [00:00<00:00, 74.9MB/s]


In [3]:
'''Переносим все изображения в одну папку'''

source_dir = "wild-animals/Animals"
target_dir = "dataset"
test_dir = "test_dataset" # там тестовые изображения будут
test_imgs_path = ['jaguar-1337201__340.jpg',
                  'tiger-cub-tiger-cub-big-cat-64152.jpeg',
                  'lion-wild-africa-african.jpg']


if not os.path.exists(target_dir):
  os.makedirs(target_dir)

for animal_folder in os.listdir(source_dir):
  animal_path = os.path.join(source_dir, animal_folder)
  if os.path.isdir(animal_path):
    for filename in os.listdir(animal_path):
      if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        source_file = os.path.join(animal_path, filename)
        target_file = os.path.join(target_dir, filename)
        shutil.move(source_file, target_file)
        # print(f"Перемещен файл: {filename}")


'''Возьмем пару изображений, для которых будем искать похожие'''
os.makedirs(test_dir, exist_ok=True)
for filename in test_imgs_path:
  source_path = os.path.join(target_dir, filename)
  target_path = os.path.join(test_dir, filename)
  shutil.move(source_path, target_path)

print(f"Объем датасета: {len(os.listdir(target_dir))}")
# shutil.rmtree('wild-animals')
# shutil.rmtree('dataset')
# shutil.rmtree('test_dataset')

Объем датасета: 162


# 1. hog и faiss

In [4]:
'''инвариантный к повороту HOG дескриптор'''

def HOG_init(img, mode="stats"):
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy)

    bin_n = 16
    bin = np.int32(bin_n * ang / (2 * np.pi))

    bin_cells = []
    mag_cells = []

    cellx = celly = 8

    for i in range(0, int(img.shape[0] / celly)):
        for j in range(0, int(img.shape[1] / cellx)):
            temp = bin[i * celly : i * celly + celly, j * cellx : j * cellx + cellx] # берем конкретное окно
            values, counts = np.unique(temp.ravel(), return_counts=True) # считаем кол-во значений

            dict_v_c = dict(zip(values, counts))
            dict_c_v = dict(zip(counts, values))

            norm_coef = counts.max()

            if mode == "stats":
                temp_answer = []

                for k in range(bin_n):
                    if k in dict_v_c:
                        temp_answer.append(dict_v_c[k] / norm_coef)
                    else:
                        temp_answer.append(0.0)

                bin_cells.append(temp_answer)
            else:
                bin_cells.append(dict_c_v[norm_coef])

    return np.array(bin_cells).ravel()


# Функция вычисляет HOG дескриптор, инвариантный относительно поворота изображений за счет усреднения дескрипторов для num_rotates поворотов изображения.
def HOG_INV(img, num_rotates=8, mode="stats"):
    '''1. Поиск центра изображения'''
    h, w = img.shape[:2]
    img_center = (w // 2, h // 2)

    '''2. Инициализация усредненного вектора'''
    hog_sum = np.zeros((img.shape[0] // 8 * img.shape[1] // 8 * 16,), dtype=np.float32)

    '''3. Поворот изображения, вычисление HOG и добавление результата в hog_sum'''
    for i in range(num_rotates):
        angle = 360 / num_rotates * i
        rotation_matrix = cv2.getRotationMatrix2D(img_center, angle, 1)
        rotated_img = cv2.warpAffine(img, rotation_matrix, (w, h))
        cur_hog = HOG_init(rotated_img, mode)
        hog_sum += cur_hog

    '''4. Усреднение'''
    avg_hog = hog_sum / num_rotates

    return avg_hog


Перед извлечение признаков HOG изображений, создадим несколько вспомогательных функций

In [5]:
'''Загрузка и небольшая предобработка изображения'''
def load_img(image_path, size=(128, 128)):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, size)
    return img

'''Извлечение HOG для всех изображений'''
def HOG_features_extracter(dataset_path):
    hog_features, img_path = [], []
    for img_filename in tqdm(os.listdir(dataset_path)):
        image_path = os.path.join(dataset_path, img_filename)
        img = load_img(image_path)
        hog_desc = HOG_INV(img)
        hog_features.append(hog_desc)
        img_path.append(image_path)
    return np.array(hog_features, dtype=np.float32), img_path


In [6]:
'''Извлечение HOG для всех изображений'''
hog_features, img_paths = HOG_features_extracter(target_dir)

'''Извлечение HOG для тестовых изображений'''
hog_features_test, test_img_paths = HOG_features_extracter(test_dir)

'''FAISS (с L2-нормой)'''
index = faiss.IndexFlatL2(hog_features.shape[1])
index.add(hog_features)

100%|██████████| 3/3 [00:00<00:00,  4.90it/s]


### Визуализируем схожие изображения

Для этого 2 вспомогательные функции.

In [7]:
'''Для вычисления L2 расстояний'''
def calc_dists(test_embedding, similar_embeddings):
    distances = []
    for embedding in similar_embeddings:
        l2_distance = euclidean(test_embedding, embedding)
        distances.append(l2_distance)
    return distances


'''Визуализация исходного изображения и num_similar схожих'''
def visualize_results(test_img_path, img_paths, I, num_similar=3):
    # пути картинок, которые будут визуализироваться
    similar_paths = [img_paths[idx] for idx in I[0][:num_similar]]
    paths = [test_img_path]
    paths.extend(similar_paths)
    titles = ['Исходное'] + [f'Похожее № {i + 1}' for i in range(num_similar)]
    # Визуализация
    plt.figure(figsize=(15, 5))
    for i, path in enumerate(paths):
        img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        plt.subplot(1, num_similar + 1, i + 1)
        plt.imshow(img)
        plt.title(titles[i])
        plt.axis('off')
    plt.show()

In [8]:
NUM_SIMILAR = 3

for i, test_hog_vector in enumerate(hog_features_test):
    _, I = index.search(np.array([test_hog_vector]), NUM_SIMILAR)

    similar_hogs = [hog_features[idx] for idx in I[0][:NUM_SIMILAR]]  # дескрипторы HOG для топа похожих изображений
    distances = calc_dists(test_hog_vector, similar_hogs) # расстояния

    for j, idx in enumerate(I[0][:NUM_SIMILAR]):
        print(f" L2 = {distances[j]:.2f} (Похожее №{j+1})")
    visualize_results(test_img_paths[i], img_paths, I, NUM_SIMILAR)
    print('\n\n\n')

Output hidden; open in https://colab.research.google.com to view.

# 2. Претрейны CNN на imagenet и FAISS

In [9]:
# подготовка изображений к загрузке в модель resnet
preprocess = transforms.Compose([transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.485, 0.456, 0.406),
                                                         (0.229, 0.224, 0.225))])

In [10]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

'''Класс для того, чтобы извлекать эмбеддинги'''
class EmbeddingExtractor:
  def __init__(self, preprocess=preprocess, device=DEVICE):
    self.preprocess = preprocess
    self.model = resnet34(pretrained=True) # претрейн модели resnet34
    self.device = device

    # без "головы"
    self.model = torch.nn.Sequential(*(list(self.model.children())[:-1]))
    self.model = self.model.to(self.device)
    self.model.eval() # градиенты не нужны

  def get_feature(self, image_path):
    tensor_img = self.preprocess(Image.open(image_path).convert('RGB')).unsqueeze(0).to(self.device)
    with torch.no_grad():
      embedding = self.model(tensor_img).cpu().flatten().numpy()
    return embedding

In [11]:
def get_embeddings(emb_extractor, dataset_path):
    embeddings, img_path = [], []
    for img_filename in tqdm(os.listdir(dataset_path)):
        image_path = os.path.join(dataset_path, img_filename)
        img_path.append(image_path)
        embedding = emb_extractor.get_feature(image_path)
        embeddings.append(embedding)
    return np.array(embeddings), img_path

In [12]:
emb_extractor = EmbeddingExtractor()
'''эмбеддинги для всех изображений'''
embeddings, img_paths = get_embeddings(emb_extractor, target_dir)

'''эмбеддинги для тестовых изображений'''
test_embeddings, test_img_paths = get_embeddings(emb_extractor, test_dir)

'''FAISS (с L2-нормой)'''
index2 = faiss.IndexFlatL2(embeddings.shape[1])
index2.add(embeddings)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 3/3 [00:00<00:00,  4.32it/s]


Визуализация

In [13]:
NUM_SIMILAR = 3

for i, test_embedding in enumerate(test_embeddings):
    _, I = index2.search(np.array([test_embedding]), NUM_SIMILAR)
    similar_embs = [embeddings[idx] for idx in I[0][:NUM_SIMILAR]]
    distances = calc_dists(test_embedding, similar_embs)

    for j, idx in enumerate(I[0][:NUM_SIMILAR]):
        print(f" L2 = {distances[j]:.2f} (Похожее №{j+1})")
    visualize_results(test_img_paths[i], img_paths, I, NUM_SIMILAR)
    print('\n\n\n')

Output hidden; open in https://colab.research.google.com to view.

# Вывод

Искать похожие на заданную картинку изображения посредством использования эмбедингов, созданных с помощью предобученной модели ResNet34, оказалось лучше, чем с помощью HOG + faiss. На примере тестовых изображений видно, что на основе HOG были случаи, когда рекомендуются не очень релавантные изображения (вместо изображения тигра например предлагается картинка с гепардом), хотя при этом значения метрики L2 были меньше, по сравнению с другими изображениями. Или например если посмотреть на первое тестовое изображение, то на основе HOG в качестве похожих изображений были предложены картинки с животными, которые тоже смотрят влево, хотя при этом сами животные разные - второй же способ уловил, что требуется искать именно льва.